In [61]:
import numpy as np

In [62]:
AgentTypes = ["qL", "bs", "bs", "bs"] # bs=base-stock, qL=Q-Learning
time = 30 + 5 # 1st term is the per game time. 2nd term is added time to account for leadtimes
cp = [4, 3, 3, 3] #Shortage Costs per item per day
ch = [3, 2, 2, 2] #Holding Costs per item per day

# Information and transportation lead times
# leadtimes[0] is the lead time between agent 0 and 1.
leadtimes = [1,1,2,1] 


base_stock = 5 # base-stock level
number_of_games = 1000

#Action space = {0,1,2,3}

In [63]:
#Generate Random Demand
def generate_demand():
  demand = []
  for i in range(time):
    demand.append(np.random.randint(0,3))
  return demand  
#print(demand)


In [64]:
#States:[ [-inf,-3], -2, -1, 0, 1, 2, [3,inf] ]
def convert_to_state(IL):
  if IL<-3:
    return 0 
  elif IL>3:
    return 6
  else:
    return IL+3  


In [67]:
#Reset Game variables
IL = np.zeros((4,time), dtype=int) #Initial Inventory Levels
AS = np.zeros((4,time), dtype=int) #Initial Arriving Shipments
AO = np.zeros((4,time), dtype=int) #Initial Arrived Orders
IL[:,0]=base_stock
AO[0,:] = generate_demand()

print(IL)
print(AO)

[[5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
[[2 2 2 2 0 1 0 2 0 0 1 0 2 1 1 1 1 1 1 1 0 0 1 1 1 1 1 2 2 1 1 2 1 2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [68]:
np.random.seed(0)
Q = np.zeros((7,4))
gamma = 0.8
epsilon = 0.9


for game in range(number_of_games):
  #Reset Variables
  costs = []
  currTime=0
  i=0
  IL = np.zeros((4,time), dtype=int) 
  AS = np.zeros((4,time), dtype=int) 
  AO = np.zeros((4,time), dtype=int) 
  IL[:,0]=base_stock
  AO[0,:] = generate_demand()

  #Begin Game
  for currTime in range(time-5):
      per_day_cost = 0

      #AO[0][currTime] = np.random.randint(0,3) #Random Demand Generation

      IL[0][currTime] = IL[0][currTime] + AS[0][currTime] - AO[0][currTime] #Update Inventory
      
      state = convert_to_state(IL[0][currTime])
      order_amount = 0
      if np.random.random() < epsilon:
        order_amount = np.random.randint(0,4)
      else:
        order_amount = np.argmin( Q[state,:])

      
      AO[1][currTime + leadtimes[0]] =  order_amount #Order Amount

      IL[0][currTime+1] = IL[0][currTime]
      per_day_cost = per_day_cost + ch[0]*max(IL[0][currTime],0) + cp[0]*min(IL[0][currTime],0)*(-1)
    
      for i in range(1,4):
        #Dealing with backorder
        if IL[i][currTime]<0:
          AS[i-1][currTime + leadtimes[i-1]] = min( abs(IL[i][currTime]) , AS[i][currTime])

        IL[i][currTime] = IL[i][currTime] + AS[i][currTime]

        #Dealing with Demand
        AS[i-1][currTime + leadtimes[i-1]] = max( min(AO[i][currTime], IL[i][currTime]) , 0 ) 
        IL[i][currTime] = IL[i][currTime] - AO[i][currTime]
        if i<=2:
          AO[i+1][currTime + leadtimes[i]] = max(0, base_stock-IL[i][currTime])
        else:
          AS[i][currTime + leadtimes [i]] =  max(0, base_stock-IL[i][currTime])
        IL[i][currTime+1] = IL[i][currTime]
        per_day_cost = per_day_cost + ch[i]*max(IL[i][currTime],0) + cp[i]*min(IL[i][currTime],0)*(-1)
      
      #print("currTime = ",currTime)
      #print("per_day_cost = ",per_day_cost)
      
      
      costs.append(per_day_cost)
      temp_cost = costs[max(currTime-2,0) :]
      reward = sum(temp_cost)/len(temp_cost)
      next_state = convert_to_state(IL[0][currTime+1] + AS[0][currTime+1] - AO[0][currTime+1])

      Q[state,order_amount] = reward + gamma * min(Q[next_state,:])
      #print(Q)
      #print("reward = ",reward)
      #IL[:,currTime+1] = IL[:,currTime]
      #print("IL = ",AS[0][currTime])
      #print("AO = ",AO)
  if game%10==0: 
    print("Game Number: ",game)
    print(costs)
    print(IL)
    print(AO)
    print(AS)
    print(Q)


Game Number:  0
[45, 42, 42, 35, 28, 35, 44, 44, 44, 51, 60, 60, 66, 87, 114, 153, 198, 256, 326, 405, 480, 565, 655, 754, 860, 973, 1096, 1233, 1368, 1522]
[[   5    4    4    3    4    5    5    4    4    7    8    6    5    3
     1    1    0   -1   -2   -3   -3   -4   -4   -4   -5   -7   -7   -9
    -9  -10  -10    0    0    0    0]
 [   5    5    5    3    0    2    4    4    1    0   -3   -3   -3   -6
    -8  -10  -10  -10  -12  -15  -15  -15  -15  -18  -19  -20  -23  -24
   -25  -27  -27    0    0    0    0]
 [   5    5    5    5    3   -2   -5   -6   -5   -4   -4   -7  -10  -13
   -19  -27  -37  -47  -57  -69  -84  -99 -114 -129 -147 -166 -186 -209
  -233 -258 -258    0    0    0    0]
 [   5    5    5    5    5    5    3   -2   -5   -6   -5   -4   -4   -7
   -10  -13  -19  -27  -37  -47  -57  -69  -84  -99 -114 -129 -147 -166
  -186 -209 -209    0    0    0    0]]
[[  0   1   0   1   1   2   0   2   0   0   0   2   1   2   2   0   1   1
    1   1   0   1   0   0   1   2   0   

In [ ]:
print(max(IL[1,:]))

6
